In [1]:
import os
os.chdir("/app")

In [2]:
import pandas as pd
from pathlib import Path
from nbgrader.api import Gradebook
from nbgrader.apps import NbGraderAPI
from traitlets.config import Config
from tqdm.auto import tqdm
import re
import os
import time

In [3]:
COURSE_ID = "itmo_recsys_2025_spring"

In [11]:
config = Config()
config.CourseDirectory.course_id = COURSE_ID
# config.ExecutePreprocessor.environment = {
#     'DATA_PATH_2': '/path/to/data'
# }
config.ExecutePreprocessor.timeout = 1800

In [12]:
nbg = NbGraderAPI(config=config)
gradebook = nbg.gradebook

In [6]:
ASSIGNMENT = "hw_3"
NOTEBOOK = "knn"  # without .ipynb
MAX_SCORE = 15

In [7]:
os.environ["DATA_PATH"] = "/usr/local/share/nbgrader/exchange/data/data_original"

In [ ]:
1MaxOn1
AlViKl
Kraagger-del
limmark21
nbelyaev1
nikiduki

In [40]:
student_ids = {s["id"] for s in nbg.get_students()}
sorted(student_ids - nbg.get_autograded_students(ASSIGNMENT))

['1MaxOn1',
 'AlViKl',
 'Kraagger-del',
 'ONEPANTSU',
 'limmark21',
 'nbelyaev1',
 'nikiduki']

In [42]:
sorted(student_ids - nbg.get_submitted_students(ASSIGNMENT))

['1MaxOn1', 'AlViKl', 'Kraagger-del', 'limmark21', 'nbelyaev1', 'nikiduki']

In [39]:
nbg.get_students()

[{'id': 'koshkidadanet',
  'first_name': 'Денис Вадимович',
  'last_name': 'Агапитов',
  'email': None,
  'score': 22.0,
  'max_score': 30.0,
  'lms_user_id': None},
 {'id': 'n0tmyself',
  'first_name': 'Дмитрий Владимирович',
  'last_name': 'Альшевский',
  'email': None,
  'score': 25.0,
  'max_score': 30.0,
  'lms_user_id': None},
 {'id': 'Kraagger-del',
  'first_name': 'Семён Дмитриевич',
  'last_name': 'Балицкий',
  'email': None,
  'score': 4.0,
  'max_score': 30.0,
  'lms_user_id': None},
 {'id': 'nbelyaev1',
  'first_name': 'Никита Игоревич',
  'last_name': 'Беляев',
  'email': None,
  'score': 15.0,
  'max_score': 30.0,
  'lms_user_id': None},
 {'id': 'nbusko',
  'first_name': 'Никита Алексеевич',
  'last_name': 'Бусько',
  'email': None,
  'score': 28.0,
  'max_score': 30.0,
  'lms_user_id': None},
 {'id': 'alexbuyan',
  'first_name': 'Александр Алексеевич',
  'last_name': 'Буянтуев',
  'email': None,
  'score': 28.0,
  'max_score': 30.0,
  'lms_user_id': None},
 {'id': 'ONEPA

# Collect

In [9]:
nbg.collect(ASSIGNMENT, update=False)

{'success': True,
 'log': "[INFO] Processing 41 submissions of 'hw_3' for course 'itmo_recsys_2025_spring'\n[WARNING] \n    /usr/local/share/nbgrader/exchange/itmo_recsys_2025_spring/inbound/smalda+hw_3+2025-04-03T05:42:09 UTC+00 claims to be submitted by smalda but is owned by dev; cheating attempt?\n    you may disable this warning by unsetting the option CollectApp.check_owner\n    \n[INFO] Collecting submission: smalda hw_3\n[WARNING] \n    /usr/local/share/nbgrader/exchange/itmo_recsys_2025_spring/inbound/ElenaSuhova+hw_3+2025-04-03T05:42:43 UTC+00 claims to be submitted by ElenaSuhova but is owned by dev; cheating attempt?\n    you may disable this warning by unsetting the option CollectApp.check_owner\n    \n[INFO] Collecting submission: ElenaSuhova hw_3\n[WARNING] \n    /usr/local/share/nbgrader/exchange/itmo_recsys_2025_spring/inbound/qw1zzard+hw_3+2025-04-03T05:42:19 UTC+00 claims to be submitted by qw1zzard but is owned by dev; cheating attempt?\n    you may disable this war

# Autograde

In [20]:
autograded_student_ids = nbg.get_autograded_students(ASSIGNMENT)
scores = {
    s_id: gradebook.find_submission_notebook(NOTEBOOK, ASSIGNMENT, s_id).score
    for s_id in autograded_student_ids
}
len({s_id: score for s_id, score in scores.items() if score < MAX_SCORE})

6

In [21]:
for student_id, score in scores.items():
    if score == MAX_SCORE:
        continue
    print(f"{student_id}: {score}")
    
    nb_path = Path("autograded") / student_id / ASSIGNMENT / f"{NOTEBOOK}.ipynb"
    nb_text = nb_path.read_text()

    if "CellTimeoutError" in nb_text:
        print("- CellTimeoutError")
        
        
    elif "ModuleNotFound" in nb_text:
        print("  - Module not found")
        for match in re.findall(r"ModuleNotFound.+", nb_text):
            print(f"    - {match}")

    elif "NameError" in nb_text:
        print("  - NameError")
        for match in re.findall(r"NameError.+not defined", nb_text):
            print(f"    - {match}")

    print("=" * 50)
    print()

martynov-dm: 12.0

popov101: 10.0

BulatMaratovich: 7.0
  - NameError
    - NameError\u001b: name 'interactions' is not defined
    - NameError\u001b: name 'd' is not defined
    - NameError\u001b: name 'reco_hot' is not defined
    - NameError\u001b: name 'reco_hot' is not defined
    - NameError\u001b: name 'd' is not defined
    - NameError\u001b: name 'd' is not defined

Qeshtir: 13.0

smalda: 12.0

nonodoubt: 12.0



In [13]:
student_ids_requested_to_grade = [
    "nonodoubt",  # timeout
    "stasstaf",  # timeout
    "V-slav-github",  # timeout
]
# student_ids_requested_to_grade = None

In [14]:
submitted_student_ids = nbg.get_submitted_students(ASSIGNMENT)
type(submitted_student_ids), len(submitted_student_ids)

(set, 41)

In [15]:
autograded_student_ids = nbg.get_autograded_students(ASSIGNMENT)
type(autograded_student_ids), len(autograded_student_ids)

(set, 40)

In [16]:
not_autograded_student_ids = list(submitted_student_ids - autograded_student_ids)
len(not_autograded_student_ids)

1

In [17]:
if student_ids_requested_to_grade is not None:
    student_ids_to_grade = student_ids_requested_to_grade
    force = True
else:
    student_ids_to_grade = not_autograded_student_ids
    force = False

In [18]:
f"{force=}", student_ids_to_grade

('force=True', ['nonodoubt', 'stasstaf', 'V-slav-github'])

In [19]:
results = {}
for student_id in tqdm(student_ids_to_grade):
    ts = time.time()
    res = nbg.autograde(assignment_id=ASSIGNMENT, student_id=student_id, force=force, create=False)
    elapsed = time.time() - ts
    res["time"] = elapsed
    results[student_id] = res

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# successes = [res for res in results if res["success"]]
# failures = [res for res in results if not res["success"]]

# graded = [res for res in successes if "[INFO] Autograding" in res["log"]]
# skipped = [res for res in successes if res["log"].startswith("[INFO] Skipping")]


# print("Total: ", len(results))
# print("failures: ", len(failures))
# print("successes: ", len(successes))
# print("graded: ", len(graded))
# print("skipped: ", len(skipped))

# Get per-task scores

In [26]:
student_ids_requested_to_grade = None

In [27]:
if student_ids_requested_to_grade is not None:
    students_to_get_scores = student_ids_requested_to_grade
else:
    students_to_get_scores = nbg.get_autograded_students(ASSIGNMENT)

In [28]:
scores = {}
for student_id in tqdm(students_to_get_scores):
    nb = gradebook.find_submission_notebook(NOTEBOOK, ASSIGNMENT, student_id)
    student_scores = {
        "total": {"actual": nb.score, "max": nb.max_score},
        "cells": [
            {"actual": g.score, "max": g.max_score}
            for g in nb.grades
        ]
    }
    scores[student_id] = student_scores

  0%|          | 0/40 [00:00<?, ?it/s]

In [29]:
scores

{'dimajyg': {'total': {'actual': 15.0, 'max': 15.0},
  'cells': [{'actual': 3.0, 'max': 3.0},
   {'actual': 2.0, 'max': 2.0},
   {'actual': 2.0, 'max': 2.0},
   {'actual': 3.0, 'max': 3.0},
   {'actual': 5.0, 'max': 5.0}]},
 'DmitryRedko': {'total': {'actual': 15.0, 'max': 15.0},
  'cells': [{'actual': 3.0, 'max': 3.0},
   {'actual': 2.0, 'max': 2.0},
   {'actual': 2.0, 'max': 2.0},
   {'actual': 3.0, 'max': 3.0},
   {'actual': 5.0, 'max': 5.0}]},
 'PaulRychkov': {'total': {'actual': 15.0, 'max': 15.0},
  'cells': [{'actual': 3.0, 'max': 3.0},
   {'actual': 2.0, 'max': 2.0},
   {'actual': 2.0, 'max': 2.0},
   {'actual': 3.0, 'max': 3.0},
   {'actual': 5.0, 'max': 5.0}]},
 'n0tmyself': {'total': {'actual': 15.0, 'max': 15.0},
  'cells': [{'actual': 3.0, 'max': 3.0},
   {'actual': 2.0, 'max': 2.0},
   {'actual': 2.0, 'max': 2.0},
   {'actual': 3.0, 'max': 3.0},
   {'actual': 5.0, 'max': 5.0}]},
 'LeftGoga': {'total': {'actual': 15.0, 'max': 15.0},
  'cells': [{'actual': 3.0, 'max': 3.0},

# Make human-readable feedback

In [30]:
CELL_NAMES = [
    "тестовая выборка",  
    "правильность холодных", 
    "метрики холодных", 
    "правильность горячих", 
    "метрики горячих",
]

**Скоры предполагаются целыми**

In [31]:
feedbacks = []
for student_id, student_scores in scores.items():
    total_score = student_scores["total"]
    feedback_scores = [f"total: {total_score['actual']:.0f} / {total_score['max']:.0f}"]
    for cell_name, cell_score in zip(CELL_NAMES, student_scores["cells"]):
        feedback_scores.append(f"{cell_name}: {cell_score['actual']:.0f} / {cell_score['max']:.0f}")
    feedback = {
        "student_id": student_id,
        "feedback": ", ".join(feedback_scores),
        "score": round(total_score["actual"])
    }
    feedbacks.append(feedback)

In [32]:
feedbacks

[{'student_id': 'dimajyg',
  'feedback': 'total: 15 / 15, тестовая выборка: 3 / 3, правильность холодных: 2 / 2, метрики холодных: 2 / 2, правильность горячих: 3 / 3, метрики горячих: 5 / 5',
  'score': 15},
 {'student_id': 'DmitryRedko',
  'feedback': 'total: 15 / 15, тестовая выборка: 3 / 3, правильность холодных: 2 / 2, метрики холодных: 2 / 2, правильность горячих: 3 / 3, метрики горячих: 5 / 5',
  'score': 15},
 {'student_id': 'PaulRychkov',
  'feedback': 'total: 15 / 15, тестовая выборка: 3 / 3, правильность холодных: 2 / 2, метрики холодных: 2 / 2, правильность горячих: 3 / 3, метрики горячих: 5 / 5',
  'score': 15},
 {'student_id': 'n0tmyself',
  'feedback': 'total: 15 / 15, тестовая выборка: 3 / 3, правильность холодных: 2 / 2, метрики холодных: 2 / 2, правильность горячих: 3 / 3, метрики горячих: 5 / 5',
  'score': 15},
 {'student_id': 'LeftGoga',
  'feedback': 'total: 15 / 15, тестовая выборка: 3 / 3, правильность холодных: 2 / 2, метрики холодных: 2 / 2, правильность горячи

In [49]:
df = pd.DataFrame(feedbacks)
df

,student_id,feedback,score
0,dimajyg,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
1,DmitryRedko,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
2,PaulRychkov,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
3,n0tmyself,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
4,LeftGoga,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
5,Postironia1,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
6,khrstln,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
7,V-slav-github,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
8,nbusko,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
9,GlebIsrailevich,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15


In [35]:
df["feedback"][0]

'total: 9 / 15, precision_and_recall: 1 / 1, ap_at_3_for_user_2: 1 / 1, map_at_3: 1 / 1, dcg_at_3_for_user_2: 1 / 1, idcg_at_3_for_user_2: 1 / 1, ndcg_at_3: 0 / 1, weighted_recall_correct: 0 / 2, weighted_recall_efficient: 0 / 3, model_A: 2 / 2, model_B: 2 / 2'

In [46]:
students_df = pd.read_csv("/usr/local/share/nbgrader/exchange/itmo_recsys_2025_spring/students.tsv", sep="\t")

students_df = students_df.loc[students_df["Github repo"].notnull()].copy()
students_df["student_id"] = students_df["Github repo"].str.removeprefix("https://github.com/").str.split("/", expand=True)[0]
students_df = students_df[["ФИО", "student_id"]]
students_df.head()

,ФИО,student_id
1,Горбунов Никита Сергеевич,nsgorbunov
2,Ильин Глеб Романович,GlebIsrailevich
3,Клакевич Александр Викторович,AlViKl
4,Латыпов Булат Маратович,BulatMaratovich
6,Попов Владимир Алексеевич,popov101


In [50]:
student_id_to_name = students_df.set_index("student_id")["ФИО"]
df.insert(0, "ФИО", df["student_id"].map(student_id_to_name))
df.head()

,ФИО,student_id,feedback,score
0,Тихановский Дмитрий Александрович,dimajyg,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
1,Редько Дмитрий Александрович,DmitryRedko,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
2,Рычков Павел Андреевич,PaulRychkov,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
3,Альшевский Дмитрий Владимирович,n0tmyself,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15
4,Поляков Егор Сергеевич,LeftGoga,"total: 15 / 15, тестовая выборка: 3 / 3, прави...",15


In [51]:
df.to_csv("hw_3_feedback.tsv", sep="\t", index=False)